# QR decomposition

Alberto Quaini

### Import some libraries

In [2]:
import numpy as np
from scipy import linalg as la

## Problem 1

In [3]:
def mod_gram_schmidt(A):
    """ Compute the modified Gram-Schmidt QR decomposition of A
    Input: A, array(m, n)
    Output: Q, array(m, n) orthonormal
            R, array(n, n) upper triangular
    """
    
    m, n = A.shape
    r = np.linalg.matrix_rank(A)
    if r < n:
        print("A is not full column rank")
    Q = np.copy(A)
    R = np.zeros((n, n))
    
    for i in range(n):
        R[i,i] = la.norm(Q[:,i])
        Q[:,i] = Q[:,i] / R[i,i]
        for j in range(i+1, n):
            R[i,j] = np.dot(np.transpose(Q[:,j]), Q[:,i])
            Q[:,j] = Q[:,j] - R[i,j] * Q[:,i]
    
    return Q, R

In [4]:
A = np.random.random((6,4))
Q, R = mod_gram_schmidt(A)

In [11]:
print("Is R upper triangular? ", np.allclose(np.triu(R), R))
print("Is Q orthonormal? ", np.allclose(Q.T @ Q, np.identity(4)))
print("Is true that A = QR? ", np.allclose(Q @ R, A))
Q1, R1 = la.qr(A, mode="economic")
print("Is Q close to Q1? ", np.allclose(np.abs(Q), np.abs(Q1)))
print("Is R close to R1? ", np.allclose(np.abs(R), np.abs(R1)))

Is R upper triangular?  True
Is Q orthonormal?  True
Is true that A = QR?  True
Is Q close to Q1?  True
Is R close to R1?  True


## Problem 2

In [13]:
def det_QR(A):
    """ Compute the determinant of invertible matrix A
    via QR decomposition
    """
    
    m, n = A.shape
    r = np.linalg.matrix_rank(A)
    if not n == m:
        print("A is not square")
    if r < n:
        print("A is not invertible")
    
    Q, R = la.qr(A, mode="economic")
    return abs(np.prod(np.diag(R)))

In [14]:
A = np.random.random((6,6))
r = round(det_QR(A), 4)
print("Rank of A is: ", r)
r1 = round(abs(la.det(A)), 4)
print("Rank of A via scipy linear algebra package is: ", r1)

Rank of A is:  0.0068
Rank of A via scipy linear algebra package is:  0.0068


Function in one line:

In [15]:
det_oneline = lambda A: abs(np.prod(np.diag(la.qr(A, mode='economic')[1])))

In [16]:
# Test
r2 = round(det_oneline(A), 4)
print("Rank of A from one-line function is: ", r2)

Rank of A from one-line function is:  0.0068


## Problem 3

In [17]:
def lin_sys(A, b):
    """ Solve a linear system
    """
    
    m, n = A.shape
    l = len(b)
    r = np.linalg.matrix_rank(A)
    if not n == m:
        print("A is not square")
    if r < n:
        print("A is not invertible")
    if not l == n:
        print("b is not of the right length")
        
    Q, R = la.qr(A, mode = 'economic')
    y = np.dot(np.transpose(Q), b)
    x = np.zeros(n)

    for j in range(n-1, -1, -1):
        x[j] = (y[j] - np.dot(R[j, j+1:], x[j+1:])) / R[j,j]
    
    return x

In [18]:
A = np.random.random((4, 4))
b = np.random.random(4)
x = lin_sys(A, b)
x1 = np.linalg.solve(A, b)
print("Is the result of lin_sys the same as numpy linalg 'solve'?", np.allclose(x, x1))

Is the result of lin_sys the same as numpy linalg 'solve'? True


## Problem 4

In [19]:
sign = lambda x: 1 if x >= 0 else -1

In [20]:
def householder(A):
    """ Compute QR decomposition of A via householder method
    """
    
    m, n = A.shape
    r = np.linalg.matrix_rank(A)
    if r < n:
        print("A is not full column rank")
    
    R = np.copy(A)
    Q = np.eye(m)
    
    for k in range(n):
        u = np.copy(R[k:,k])
        u[0] = u[0] + sign(u[0]) * la.norm(u)
        u = u / la.norm(u)
        R[k:,k:] = R[k:,k:] - 2 * np.outer(u, np.dot(np.transpose(u), R[k:,k:]))
        Q[k:,:] = Q[k:,:] - 2 * np.outer(u, np.dot(np.transpose(u), Q[k:,:]))
    
    return np.transpose(Q), R

In [21]:
A = np.random.random((5, 3))
Q,R = householder(A) 
print("Is R upper triangular? ", np.allclose(np.triu(R), R))
print("Is Q orthonormal? ", np.allclose(Q.T @ Q, np.identity(5)))
print("Is true that A = QR? ", np.allclose(Q @ R, A))

Is R upper triangular?  True
Is Q orthonormal?  True
Is true that A = QR?  True


## Problem 5

In [22]:
def hessenberg(A):
    """ Compute Hessenberg QR decomposition of A
    """
    
    m, n = A.shape
    r = np.linalg.matrix_rank(A)
    if not n == m:
        print("A is not square")
    if r < n:
        print("A is not invertible")
        
    H = np.copy(A)
    Q = np.eye(m)
    
    for k in range(n-2):
        u = np.copy(H[k+1:,k])
        u[0] = u[0] + sign(u[0]) * la.norm(u)
        u = u / la.norm(u)
        H[k+1:,k:] = H[k+1:,k:] - 2 * np.outer(u, np.dot(np.transpose(u), H[k+1:,k:]))
        H[:,k+1:] = H[:,k+1:] - 2 * np.outer(np.dot(H[:,k+1:], u), np.transpose(u))
        Q[k+1:,:] = Q[k+1:,:] - 2 * np.outer(u, np.dot(np.transpose(u), Q[k+1:,:]))
        
    return H, np.transpose(Q)

In [23]:
A = np.random.random((8,8))
H, Q = hessenberg(A)
print("H has all zeros below the first subdiagonal? ", np.allclose(np.triu(H, -1), H))
print("Is true that QHQ^T = A? ", np.allclose(Q @ H @ Q.T, A))

H has all zeros below the first subdiagonal?  True
Is true that QHQ^T = A?  True
